In [ ]:
#!pip install -q diffusers transformers accelerate peft
#!pip install scipy
#!pip install compel

In [ ]:
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, DiffusionPipeline, StableDiffusionImg2ImgPipeline,StableDiffusionImageVariationPipeline
import torch
pipeline = StableDiffusionPipeline.from_pretrained("diffusers/examples/research_projects/multi_subject_dreambooth/HoPla_sug_20_full", torch_dtype=torch.float16, use_safetensors=True,force_download=True).to("cuda")

In [ ]:
pipeline.scheduler.compatibles

In [ ]:
from diffusers import (
    DDPMScheduler,
    DDIMScheduler,
    PNDMScheduler,
    LMSDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
    EulerDiscreteScheduler,
    HeunDiscreteScheduler,
    DPMSolverMultistepScheduler,
    DPMSolverSinglestepScheduler,
    KDPM2AncestralDiscreteScheduler
)

# Uncomment one of the following lines to set the desired scheduler for the pipeline
# pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
# pipeline.scheduler = LMSDiscreteScheduler.from_config(pipeline.scheduler.config) # Best for multiple generations
# pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
# pipeline.scheduler = EulerDiscreteScheduler.from_config(pipeline.scheduler.config) # For multiple weed generation, cross attention 1.5
pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeline.scheduler.config) # Best for small objects
# pipeline.scheduler = PNDMScheduler.from_config(pipeline.scheduler.config)
# pipeline.scheduler = HeunDiscreteScheduler.from_config(pipeline.scheduler.config)
# pipeline.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipeline.scheduler.config) # Better for cottonweed
# pipeline.scheduler = DPMSolverSinglestepScheduler.from_config(pipeline.scheduler.config)

In [ ]:
from compel import Compel
"""
This script initializes a Compel object for text-to-image generation.

Classes:
    Compel: A class from the compel library used for processing text prompts.

Variables:
    compel_proc: An instance of the Compel class, initialized with a tokenizer and text encoder from the pipeline.

Usage:
    The compel_proc object can be used to process text prompts for generating images.
"""
compel_proc = Compel(tokenizer=pipeline.tokenizer, text_encoder=pipeline.text_encoder)

In [ ]:
prompt = "A photo of HoPla Plot Planted with HoPla Plants and Weeds, High quality images, realistic "
prompt_embeds = compel_proc(prompt) #Randomized view #easyy #best prompt

In [ ]:
prompt = "A Photo of HoPla Plot Planted with HoPla Fallopia, High quality images, realistic "
prompt_embeds = compel_proc(prompt) #single plant types

In [ ]:
pipeline.enable_freeu(s1=0.6, s2=0.2, b1=1.0, b2=0.4)  # Sometimes it is good to control color

In [ ]:
pipeline.enable_freeu(s1=1.0, s2=0.5, b1=1.3, b2=1.7)

In [ ]:

import random
num_images = 10
gen_images = []

# Generate images
for i in range(num_images):
    # Generate an image using the pipeline with specified parameters
    image = pipeline(
        prompt_embeds=prompt_embeds,
        negative_prompt=("macro shot, large plants, distorted perspective, blurry, overexposed, low quality, cluttered background"),
        num_inference_steps=55,
        guidance_scale=15.5,
        cross_attention_kwargs={'scale': 0.6},
        generator=random.seed(1333333333333333227),
        cfg_rescale=5,
        height=512,
        width=512
    ).images[0]
    
    # Append the generated image to the list
    gen_images.append(image)
    
    # Save each generated image with a specific filename
    for i, img in enumerate(gen_images):
        img.save(f"Gen_Img/20_/sug_{i + 1}.png")  #output directory